## Data Cleaning and Preprocessing Notebook

## List of cleaning and Preproccessing
- This steps are decided based on the Summary of Eda notebook

1. Replace '?' with NaN

2. Drop Irrelevant or High-Null Columns:
    - identifier columns : `encounter_id`, `patient_nbr`

    - Droping columns with >80% missing values: `weight` (≈98% missing)

    - Droping `examide`, `citoglipton` as they have same value for all the records
    
    - keeping `max_glu_Serum` and `A1Cresult` (impute with 'None')

3. Handle Missing Values:
    - Categorical (Low Null) `race` :  Fill with mode (most frequent) 
    
    - Categorical (High Null but meaningful) `max_glu_serum`, `A1Cresult` :  Fill with `'None'`             
    
    - Categorical (nominal) `payer_code`, `medical_specialty`:  Fill with `'Unknown'`          
    
    - Diagnosis Columns `diag_1`, `diag_2`, `diag_3` :  Fill with `'Unknown'`    

4. Encode Categorical Variables:
    - Binary	: Map manually	`readmitted` = {'NO': 0, '>30': 0, '<30': 1}
    
    - Nominal	: Label Encoding	`gender`, `race`, `medical_specialty`, `payer_code`, `max_glu_serum`, `A1Cresult`
    
    - Medication-related : Label Encoding (as values are ordinal-like: No < Down < Steady < Up)

5. Outlier Handling:
    - `time_in_hospital`,`num_lab_procedures`,`num_procedures`,`num_medications`,
    `number_outpatient`,`number_emergency`,`number_inpatient`,`number_diagnoses`,
    `total_medications`,`num_med_changes`

    - log transformation

6. Scalling of numerical columns (dynamic for different models)

7. Feature Engineering:
    - Discretization of `age` : [0-10) → 5, [10-20) → 15 

    - Categorization `diag_1 , diag_2, diag_3` based on codes which relates to disease.

In [1]:
# System path setup
import sys
sys.path.append("..")

In [2]:
# importing packages
import pandas as pd
from src.preprocess_pipeline import preprocess

In [3]:
raw_data_path = "../data/raw/diabetic_data.csv"
df = pd.read_csv(raw_data_path)

In [4]:
# List of medication columns to be processed
medication_columns = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
    'rosiglitazone', 'acarbose', 'miglitol', 'insulin','tolazamide',
    'glyburide-metformin','acetohexamide',
    'tolbutamide',
    'troglitazone',
    'glipizide-metformin',
    'glimepiride-pioglitazone',
    'metformin-rosiglitazone',
    'metformin-pioglitazone'
]

# list of columns to label encode
label_encode_columns = ['gender', 'race', 'medical_specialty', 'payer_code', 'max_glu_serum', 'A1Cresult', 'change', 'diabetesMed']

# List of columns to log transform
log_transform_columns = ['time_in_hospital','num_lab_procedures','num_procedures','num_medications',
    'number_outpatient','number_emergency','number_inpatient','number_diagnoses',
    'total_medications','num_med_changes'
]

In [5]:
preprocessed_df = preprocess(df, medication_columns, label_encode_columns, log_transform_columns)
preprocessed_df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,diag_2_cat_Heart Failure,diag_2_cat_Hypertension,diag_2_cat_Other,diag_3_cat_Heart Disease,diag_3_cat_Heart Failure,diag_3_cat_Hypertension,diag_3_cat_Other,total_medications,num_med_changes,numeric_age
0,2,0,[0-10),6,25,1,0.693147,16,37,3.737670,...,0,0,1,0,0,0,1,3.091042,0.0,5
1,2,0,[10-20),1,1,7,1.386294,16,71,4.094345,...,0,0,0,0,0,0,1,3.091042,0.0,15
2,0,0,[20-30),1,1,7,1.098612,16,71,2.484907,...,0,0,0,0,0,0,1,3.091042,0.0,25
3,2,1,[30-40),1,1,7,1.098612,16,71,3.806662,...,0,0,0,0,0,0,1,3.091042,0.0,35
4,2,1,[40-50),1,1,7,0.693147,16,71,3.951244,...,0,0,1,0,0,0,0,3.091042,0.0,45


In [7]:
import os 
preprocessed_dir = "../data/processed"
os.makedirs(preprocessed_dir, exist_ok=True)
path = os.path.join(preprocessed_dir, "preprocessed_data.csv")
preprocessed_df.to_csv(path, index=False)